In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text
import os

In [2]:
conn = sqlite3.connect(r"C:\Users\dhanu\OneDrive\Desktop\Repo\VR_Training\module_2\database\VR.db")

In [4]:
employees_df = pd.read_sql("SELECT * FROM employees", conn)
departments_df = pd.read_sql("SELECT * FROM departments", conn)
projects_df = pd.read_sql("SELECT * FROM projects", conn)
employee_projects_df = pd.read_sql("SELECT * FROM employee_projects", conn)

In [5]:
query1 = (employees_df
    .merge(departments_df, on="department_id", how="inner")
)[["first_name","last_name","salary","department_name","location"]]


In [6]:
query2 = (employees_df
    .merge(departments_df, on="department_id", how="inner")
    .merge(employee_projects_df, on="employee_id", how="inner")
    .merge(projects_df, on="project_id", how="inner")
)

query2["budget_utilization"] = (query2["budget_y"] / query2["budget_x"]) * 100

final = query2.rename(columns={"budget_y":"project_budget", "budget_x":"depart_budget"})[
    ["first_name","last_name","department_name","project_name",
     "role","hours_allocated","project_budget","depart_budget","budget_utilization"]
]


In [7]:
result = (employees_df
    .merge(departments_df, on="department_id", how="inner")
    .loc[lambda df: df["salary"] > 75000]
    .sort_values("salary", ascending=False)
)[["first_name", "last_name", "salary", "department_name"]]


In [8]:
result = employees_df.merge(
    departments_df,
    on="department_id",
    how="left"
)[["first_name", "last_name", "salary", "department_name", "location"]]


In [9]:
result = (employees_df
    .merge(employee_projects_df, on="employee_id", how="left")
    .loc[lambda df: df["project_id"].isna()]
)[["first_name", "last_name", "department_id", "project_id"]]


In [10]:
result = (departments_df
    .merge(employees_df, on="department_id", how="left")
    .groupby(["department_id", "department_name"], as_index=False)
    .agg(employee_count=("employee_id","count"),
         avg_salary=("salary","mean"))
)


In [11]:
result = departments_df.merge(
    employees_df,
    on="department_id",
    how="left"
)[["first_name", "last_name", "department_name"]]


In [12]:
result = employees_df.merge(
    departments_df,
    on="department_id",
    how="outer"
)[["first_name", "last_name", "department_name"]]


In [13]:
result = employees_df.merge(departments_df, how="cross")[["first_name","last_name","department_name"]].head(20)


In [14]:
result = employees_df.merge(
    employees_df,
    left_on="manager_id",
    right_on="employee_id",
    suffixes=("_emp", "_mgr"),
    how="left"
)

final = result.assign(
    employee_name = result["first_name_emp"] + " " + result["last_name_emp"],
    manager_name  = result["first_name_mgr"].fillna("") + " " + result["last_name_mgr"].fillna("")
)[["employee_name","salary_emp","manager_name","salary_mgr"]]
